In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np
import keras
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import Input, MaxPooling2D,Conv2D,Dense, Dropout, Activation,Flatten, BatchNormalization
from keras.optimizers import Adam,sgd
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback, EarlyStopping
from keras.models import model_from_json
import h5py
import tensorflow as tf

Using TensorFlow backend.


In [0]:
os.listdir()
os.chdir('gdrive/My Drive/MyMIR')
os.listdir()
os.chdir('final_Xtrain')

In [0]:
os.chdir('../final_Xtrain')

In [0]:
Xtrain = []
Ytrain = []
Xtest = []
Ytest = []
Xval = []
Yval = []

In [0]:
#Xtrain = np.load("data1.npy")
#Xtest = np.load("dataB.npy")
Xval = np.load("dataC.npy")
os.chdir('../final_Ytrain')
#Ytrain = np.load("Y1.npy")
#Ytest = np.load("YB.npy")
Yval = np.load("YC.npy")

In [0]:
#Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
#print(Xtrain.shape)
#Xtest = Xtest.reshape(Xtest.shape[0],Xtest.shape[1],Xtest.shape[2],1)
#print(Xtest.shape)
Xval = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
print(Xval.shape)

(1825, 128, 1255, 1)


In [0]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [0]:
my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]

In [0]:
model = Sequential()
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) 
model.add(Dense(200,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(120,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(50,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',auc_roc])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

ValueError: ignored

In [0]:
os.listdir()
os.chdir('../final_Xtrain')

In [0]:
for i in range(0,12):
  if i<10:
    Xtrain = np.load('data'+str(i)+'.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('Y'+str(i)+'.npy')
    os.chdir('../final_Xtrain')
  elif i == 10:
    Xtrain = np.load('dataA.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YA.npy')
    os.chdir('../final_Xtrain')
  else:
    Xtrain = np.load('dataB.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YB.npy')
    os.chdir('../final_Xtrain')
  Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
  Xval = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
  model.fit(Xtrain, Ytrain,batch_size=32,epochs=10,verbose=1,validation_split=0.1,validation_data = (Xval,Yval),shuffle=True)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.metrics.auc. Note that the order of the labels and predictions arguments has been switched.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.
Train on 1108 samples, validate on 1825 samples
Epoch 1/10
1108/1108 [==============================] - 34s 31ms/step - loss: 0.8795 - acc: 0.5132 - auc_roc: 0.4897 - val_loss: 0.6775 - val_acc: 0.6046 - val_auc_roc: 0.5183
Epoch 2/10
1108/1108 [==============================] - 21s 19ms/step - loss: 0.7835 - acc: 0.5594 - auc_roc: 0.

In [0]:
tempx = [['dataD.npy','YD.npy'],['dataE.npy','YE.npy'],['dataF.npy','YF.npy']]
for tple in tempx:
  Xtestp,Ytestp = tple
  os.chdir('../final_Xtrain')
  Xtest = np.load(Xtestp)
  os.chdir('../final_Ytrain')
  Ytest = np.load(Ytestp)
  Xtest = Xtest.reshape(Xtest.shape[0],Xtest.shape[1],Xtest.shape[2],1)
  predicted = model.predict(Xtest,batch_size=32)
  print(roc_auc_score(Ytest, predicted))

0.6352202259885419
0.6831728918125046
0.7038165002510265


In [0]:
 os.chdir('/content/gdrive/My Drive/MyMIR/model')

In [0]:
# load json and create model
json_file = open('modelv1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelv1.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
model_json = model.to_json()
with open("modelv9.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelv9.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
#os.chdir('/content/gdrive/My Drive/MyMIR')
os.listdir()
#model.save('/content/gdrive/My Drive/MyMIR/model/modelv1.h5')

['modelv1.json',
 'modelv1.h5',
 'modelv2.json',
 'modelv2.h5',
 'modelv4.json',
 'modelv4.h5',
 'modelv3.json',
 'modelv3.h5',
 'modelv5.json',
 'modelv5.h5',
 'modelv6.json',
 'modelv6.h5',
 'modelv7.json',
 'modelv7.h5',
 'modelv8.json',
 'modelv8.h5',
 'modelv9.json',
 'modelv9.h5']